# VoterKarma calculation
_This notebook outlines the steps for calculating the VoterKarma score presented in the Debug Politics Hackthon 1-15-16_
### v 0.1: MVP: Logistic regression defaults to score voters

In [ ]:
import pdb
import psycopg2 as pg
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LogisticRegression

In [ ]:
#Read in db access information
vk_access = {}
with open('./db_vars', 'r') as f:
    for l in f.readlines():
        vk_access[l.split('=')[0]] = l.split('=')[1].strip('\n')

### Utilities

In [ ]:
# Utilities
def date_to_years(start, end=datetime.date.today()):
    return (end - start.date()).days / 365

def convert_to_date(col):
    return pd.to_datetime(col, errors='ignore', format="%Y-%m-%d")

### Global vars

In [ ]:
#Limit number of observations to retrieve
LIMIT = 3000
#Columns in database
HEADERS = (
'id', 'dob', 'gender', 'status', 'enrollment', 'district', 'regdate', 
'idrequired', 'idmet',
'e2001_09_primary',
'e2001_11_general',
'e2005_09_primary',
'e2005_11_general',
'e2006_11_general',
'e2008_02_primary',
'e2008_11_general',
'e2009_09_primary',
'e2009_11_general',
'e2010_09_primary',
'e2010_11_general',
'e2012_06_primary',
'e2012_09_primary',
'e2012_11_general',
'e2013_09_primary',
'e2013_11_general',
'e2014_06_primary',
'e2014_11_general'
)
#Recent elections (column names)
RECENT = {'local_primary':'e2013_09_primary', 
               'local_general':'e2013_11_general', 
               'national_midterm':'e2014_11_general',
               'national_presidential': 'e2012_11_general'}

### Voter counts for weighting
This retrieves the count of the number of people that voted in the most recent elections.  This isn't used in v0.1

In [ ]:
# Voter counts
v_cnt = {}
conn = pg.connect(database = vk_access['VK_DB'], user = vk_access['VK_U'], password = vk_access['VK_PW'],
    host = vk_access['VK_HOST'], port = vk_access['VK_PORT'])
cur = conn.cursor()
cnt_sql = """
            SELECT count(*)
            from rawvoters
            where {} = TRUE
          """
for k,v in RECENT.items():
    cur.execute(cnt_sql.format(v))
    v_cnt[k] = float(cur.fetchall()[0][0])

### Data pulling
Requires connection to vk RDS.  Use `LIMIT` global to limit number of observations.  Total dataset is >3M rows

In [ ]:
conn = pg.connect(database = vk_access['VK_DB'], user = vk_access['VK_U'], password = vk_access['VK_PW'],
    host = vk_access['VK_HOST'], port = vk_access['VK_PORT'])

cur = conn.cursor()

sel = """
SELECT {}
FROM {}
""".format(', '.join(headers), 'rawvoters')

if LIMIT is not None:
    sel += " LIMIT {}".format(LIMIT)

cur.execute(sel)
df = pd.DataFrame.from_records(cur.fetchall(), columns=headers)

In [ ]:
# Set index to 'id' and drop id
df.set_index(['id'], inplace=True)
# Convert Date fields to years duration
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(convert_to_date)
df[df.select_dtypes(['datetime64[ns]']).columns] = \
        df.select_dtypes(['datetime64[ns]']).applymap(lambda x:
                date_to_years(x))
# Create categories and pivot them
df[df.select_dtypes(['object']).columns] = \
        df.select_dtypes(['object']).apply(lambda x: x.astype('category'))

# Pivot categorical variables
cat_cols = df.select_dtypes(['category']).columns
for col in cat_cols:
    dummy = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df, dummy], axis=1)

df.drop(cat_cols, axis=1, inplace=True)

### Model making and scoring

In [ ]:
# Make model
def make_model(data, target):
    y = data[target]
    X = data.drop([target], axis=1)

    mod = LogisticRegression()
    mod.fit(X, y)
    return(mod)

In [ ]:
# Score input data
def score(mod, user_data):
    pred = mod.predict_proba(user_data)[:,1]
    return(pred)

In [ ]:
target_dict = {'local_primary':'e2013_09_primary', 
               'local_general':'e2013_11_general', 
               'national_midterm':'e2014_11_general',
               'national_presidential': 'e2012_11_general'}
mod_tr = {}
scored = pd.DataFrame()

for k,v in RECENT.items():
    mod_tr[k] = make_model(df, v)
    print k, 'accuracy', mod_tr[k].score(df.drop([v],axis=1), df[v])
    scored[k] = score(mod_tr[k], df.drop([v],axis=1))
scored['raw_voter_id'] = df.index

In [ ]:
# weighted score
scored['score_w'] = 0
for k in RECENT:
    scored['score_w'] += scored[k] * (1/vk_cnt[k])

In [ ]:
# Center percentile, max = 100%
scored['score_w_scaled'] = scored['score_w'] / scored['score_w'].max()

# Sum score - used in v 0.1
scored['score_total'] = scored['local_general'] + scored['national_midterm'] + scored['national_presidential']
scored['score_total_scaled'] = scored['score_total'] / scored['score_total'].max()

### Upload to db (takes a long time outside of AWS)

In [ ]:
conn = pg.connect(database = vk_access['VK_DB'], user = vk_access['VK_U'], password = vk_access['VK_PW'],
    host = vk_access['VK_HOST'], port = vk_access['VK_PORT'])
cur = conn.cursor()

ins_cols = ['local_general', 'local_primary', 'national_presidential', 
            'national_midterm', 'raw_voter_id', 'score_w_scaled', 'score_total_scaled']

#Counter for resume upload
#Commits every 10,000 records staged
counter = 0
for ind, vals in scored.iloc[counter:][ins_cols].iterrows():
    if counter % 10000 == 0:
        print "uploaded {} records".format(counter)
        conn.commit()
    vals_str = ', '.join([str(x) for x in vals])
    rvid = vals['raw_voter_id']
    ins = """ 
    INSERT INTO {0} ({1})
    select {2}
    WHERE NOT EXISTS (SELECT * FROM {0} WHERE raw_voter_id={3})
    """.format('voter_grades', 
            ', '.join(ins_cols), 
              vals_str,
              rvid)
    cur.execute(ins)
    counter+=1
conn.commit()